# Tarea 4. Modelos de Lenguaje Estadísticos

Guillermo Segura Gómez

## Modelos de Lenguaje y Evaluación
**1. Preprocese todos los tuits de agresividad (positivos y negativos) según su intuición para construir un buen corpus para un modelo de lenguaje (e.g., solo palabras en minúscula, etc.). Agregue tokens especiales de <s> y </s> según usted considere (e.g., al inicio y final de cada tuit). Defina su vocabulario y enmascare con <unk> toda palabra que no esté en su vocabulario.**

### Modelos de Lenguaje

Un modelo de lenguaje típicamente asigna probabilidades a secuencias de palabras o genera secuencias de palabras basándose en las probabilidades aprendidas. Estas probabilidades reflejan qué tan probable es que una palabra siga a otra u otras en una secuencia. Los modelos de lenguaje pueden ser de varios tipos, incluidos los basados en reglas gramaticales, los estadísticos, y más recientemente, los basados en redes neuronales, como los modelos de lenguaje transformadores.

### Modelos de Lenguaje de N-gramas
Los modelos de lenguaje de n-gramas son un tipo específico de modelo de lenguaje estadístico que se basa en la premisa de que la probabilidad de una palabra en una secuencia depende solo de las $n-1$ palabras anteriores. Un "n-grama" es, por lo tanto, una secuencia de $n$ elementos (por ejemplo, palabras) del texto. Por ejemplo:

- Un 1-grama (o unigrama) considera cada palabra de forma aislada.
- Un 2-grama (o bigrama) considera pares de palabras consecutivas.
- Un 3-grama (o trigrama) considera tríos de palabras consecutivas, y así sucesivamente.

En un modelo de n-gramas, la probabilidad de una palabra dada está condicionada por las $n-1$ palabras anteriores. Estos modelos son relativamente simples de construir y pueden ser muy efectivos para ciertas tareas, especialmente cuando los recursos computacionales son limitados o cuando se dispone de una cantidad limitada de datos de entrenamiento. Sin embargo, los modelos de n-gramas tienen limitaciones, como la dificultad para manejar contextos más largos y el problema de la "explosión combinatoria" de posibles n-gramas a medida que $n$ aumenta, lo que puede hacer que el modelo sea menos práctico para valores grandes de $n$.

Para poder construir un modelo de n-gramas primero necesitamos pre procesar nuestro corpus para que sea mas sencillo de manejar. Por ejemplo, podemos trabajar únicamente palabras en minúscula. Además, es necesario tener delimitadores de oraciones \<s\> y \</s\> y definir un vocabulario. Cualquier palabra fuera del vocabulario le asignamos el token \<unk\>. Vamos a trabajar con los tweets de agresividad del corpus [MEX-A3T](https://sites.google.com/view/mex-a3t/home?authuser=0) con el que hemos estado trabajando.

In [9]:
# Función que extrae el texto de dos archivos. Uno el de los documentos, otro el de las etiquetas
def get_text_from_file(path_corpus, path_truth):

    tr_text = []
    tr_labels = []

    with open(path_corpus, "r") as f_corpus, open(path_truth, "r") as f_truth:
        for tweet in f_corpus:
            tr_text += [tweet]
        for label in f_truth:
            tr_labels += [label]

    return tr_text, tr_labels

In [10]:
path_text = "/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/Data/MexData/mex20_train.txt"
path_labels = "/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/Data/MexData/mex20_train_labels.txt"

path_text_val = "/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/Data/MexData/mex20_val.txt"
path_labels_val = "/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/Data/MexData/mex20_val_labels.txt"

tr_text, tr_labels = get_text_from_file(path_text, path_labels) # Importamos los datos de entrenamiento
val_text, val_labels = get_text_from_file(path_text_val, path_labels_val) # Importamos los datos de test o validación

Lo primero que tenemos que hacer es delimitar nuestro vocabulario. Limitamos solo las palabras en minúsculas. 
Además debemos agregar los tokens de inicio y final a cada documento del corpus.

In [11]:
import nltk
from nltk.tokenize import TweetTokenizer
import re

In [12]:
# Función para preprocesar los textos
def preprocess_texts(texts):
    # Convertir a minúsculas y eliminar números y caracteres especiales
    texts_lower = [re.sub(r"http\S+|[^a-zA-Z\s]", "", doc.lower()) for doc in texts]
    return texts_lower

# Función que agrega delimitadores de oraciones a un corpus
def addlimiters(texts):
    texts_limites = []
    for doc in texts:
        newText = "<s>" + doc + "</s>"
        texts_limites.append(newText)

    return texts_limites

# Preprocesar el texto
tr_text_min = preprocess_texts(tr_text)

# Agregar delimitadores de oración
tr_text_limited = addlimiters(tr_text_min)


Ahora tokenizamos el texto.

In [13]:
tokenizer = TweetTokenizer() # Inicializar tokenizer
corpus_palabras = []

for doc in tr_text_limited:
    corpus_palabras += tokenizer.tokenize(doc)

fdist = nltk.FreqDist(corpus_palabras)

print(f"El tamaño del corpus es:", len(corpus_palabras))
print(f"El tamaño del vocabulario es:", len(fdist))

El tamaño del corpus es: 94261
El tamaño del vocabulario es: 12388


Definimos una función para construir el vocabulario. Utilizamos una longitud del vocabulario de cinco mil. 

In [14]:
# Función para ordenar las frecuencias
def  SortFrecuency(freqdist):
    # List comprenhension
    aux = [(freqdist[key], key) for key in freqdist]
    aux.sort() # Ordena la lista
    aux.reverse() # Cambiar el orden

    return aux

# Ordenamos y obtenemos el vocabulario
voc = SortFrecuency(fdist)
voc = voc[:5000]
voc[:5]

[(5278, '<s>'), (5278, '</s>'), (3102, 'que'), (3095, 'de'), (2268, 'la')]

Ahora reemplazamos las palabras desconocidas con el token \<unk\>

In [15]:
# Función para reemplazar palabras fuera del vocabulario con <unk>
def replace_unknowns(tokenized_texts, vocab):
    # Reemplazar palabras que no están en el vocabulario con <unk>
    return [[word if word in vocab else '<unk>' for word in doc] for doc in tokenized_texts]

tr_text_final = replace_unknowns(corpus_palabras, voc)

In [16]:
tr_text_final[:5]

[['<unk>', '<unk>', '<unk>'],
 ['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>'],
 ['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>'],
 ['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>'],
 ['<unk>']]

**2. Entrene tres modelos de lenguaje sobre todos los tuits: $P_{unigramas}(w_1^n)$, $P_{bigramas}(w_1^n)$, $P_{trigramas}(w_1^n)$. Para cada uno proporcione una interfaz (función) sencilla para  $P_{n-grama}(w_1^n)$ y  $P_{n-grama}(w_1^n | w_{n-N+1}^{n-1})$.  Los modelos deben tener una estrategia común para lidiar con secuencias no vistas. Puede optar por un suavizamiento Laplace o un Good-Turing discounting. Muestre un par de ejemplos de como funciona, al menos uno con una palabra fuera del vocabulario.**

El modelo de n-gramas que vamos a construir es una función que recibe una cadena de caracteres y asigna la probabilidad en base a las $n-1$ palabras de las que este formada la cadena. Para ejemplificar esto, consideremos el ejemplo mostrado en el capitulo 3 del libro de Daniel Jurafsky [1].

En el capitulo 3 del libro se busca encontrar un modelo que calcule la probabilidad de que la siguiente palabra a la oración *“
its water is so transparent that* sea *the* es decir

$$
P(\text{the}|\text{its water is so transparent that})
$$

* En un modelo de unigramas se asume que la aparición de cada palabra es independiente de las palabras antes o después de ella en la secuencia. Esto significa que no se considera el contexto en el que aparece la palabra; cada palabra es tratada de forma aislada.

    La probabilidad de una secuencia de palabras $ w_1^n $ en un modelo de unigramas se calcula simplemente como el producto de las probabilidades individuales de cada palabra en la secuencia:

    $$ P_{\text{unigramas}}(w_1^n) = P(w_1) \times P(w_2) \times \ldots \times P(w_n) = \prod_{i=1}^{n} P(w_i) $$

    Donde:
    - $ P_{\text{unigramas}}(w_1^n) $ es la probabilidad de la secuencia de palabras $ w_1, w_2, \ldots, w_n $ bajo el modelo de unigramas.
    - $ P(w_i) $ es la probabilidad de la palabra individual $ w_i $.

    La probabilidad $P(w_i)$ de cada palabra individual se estima generalmente a partir del corpus de entrenamiento. La forma más simple de estimar $P(w_i)$ es usar la frecuencia relativa de la palabra en el corpus:

    $$ P(w_i) = \frac{\text{Frecuencia de } w_i}{\text{Número total de palabras en el corpus}} $$

* En el modelo de bigramas, calculamos la probabilidad según $n-1$ palabras, es decir

    $$
    P(\text{the}|\text{that})
    $$

    La suposición de que la probabilidad de una palabra depende solo en la palabra previa se conoce como **suposición de Markov**. Para calcular la probabilidad en un modelo de bigramas de una palabra $w_n$ dada una previa palabra $w_{n-1}$ calculamos la cuenta de cuentas ocasiones aparecen las palabras juntas $C(w_{n-1} w_n)$ y la normalizamos por la suma de todos los bigramas que comparten la misma palabra $w_{n-1}$

    $$
    P(w_n|w_{n-1}) = \frac{C(w_{n-1} w_n)}{ \sum_w C(w_{n-1})}
    $$

    Podemos simplificar esta ecuación, ya que la suma de todas las cuentas de los bigramas que empiezan con la misma palabra $w_{n-1}$, debe de ser igual a la cuenta de los unigramas para esta palabra $w_{n-1}$, entonces 

    $$
    P(w_n|w_{n-1}) = \frac{C(w_{n-1} w_n)}{ C(w_{n-1})}
    $$

    Para una secuencia de tres palabras $w_1, w_2, w_3$, la probabilidad bajo el modelo de bigramas sería:

    $$ P_{\text{bigramas}}(w_1, w_2, w_3) = P(w_1) \times P(w_2 | w_1) \times P(w_3 | w_2) $$


Podemos realizar el ejemplo del libro para un corpus limitado de tres frases.

In [17]:
tweets_example = ['I am Sam', 'Sam I am', ' I do not like green eggs and ham']
# Agregar delimitadores de oración
tweets_example_limited = addlimiters(tweets_example)
print(tweets_example_limited)

['<s>I am Sam</s>', '<s>Sam I am</s>', '<s> I do not like green eggs and ham</s>']


### Unigramas

Construimos un vocabulario para unigramas. El vocabulario es basicamente el mismo que hemos manejado siempre. 

In [18]:
tokenizer = TweetTokenizer() # Inicializar tokenizer
corpus_palabras_example = []

for doc in tweets_example_limited:
    corpus_palabras_example += tokenizer.tokenize(doc)

fdist_example = nltk.FreqDist(corpus_palabras_example)

print(f"El tamaño del corpus es:", len(corpus_palabras_example))
print(f"El tamaño del vocabulario es:", len(fdist_example))

El tamaño del corpus es: 20
El tamaño del vocabulario es: 12


Ahora construimos un diccionario limitado para el uso de las palabras. Por ser un ejemplo, tomaremos todo el corpus, pero para cuando utilicemos el corpus de agresividad, se vera la utilidad. 

In [19]:
# Funcion que construye un diccionario limitado a n palabras
def DicLimited(fdist, n):
    # Obtener las n palabras más comunes
    voc = fdist.most_common(n)

    # Construir el diccionario directamente a partir de las n palabras más comunes
    dict_indices = {word: freq for word, freq in voc}

    return dict_indices

Comenzamos calculando la probabilidad de las cadenas de unigramas, es decir la función para $P_{unigramas}(w_1^n)$. El objeto `fdist_example` es un objeto de la clase *nltk.probability.FreqDist*. Este objeto tiene la característica de que almacena la distribución de frecuencias del corpus tokenizado. Entonces si accedemos a el podemos obtener la frecuencia general de las palabras. 

Ademas, es necesario tener una estrategia para manejar los casos en los que tenemos una secuencia cuya alguna de las palabras no estan en el vocabulario ya que por producto de probabilidad, tendremos probabilidad cero, lo cual tiene que ser "suavizado" de alguna forma. Podemos utilizar un suavizamiento *Laplace* o un *Good-Turing discounting*.

Los suavizados de Laplace y Good-Turing son técnicas utilizadas en modelos de lenguaje, como los de n-gramas, para manejar el problema de las probabilidades cero para n-gramas no observados en el conjunto de datos de entrenamiento. Ambas técnicas ajustan las probabilidades estimadas para asignar alguna probabilidad a los n-gramas no observados, mejorando la generalización del modelo a datos no vistos previamente.

#### Suavizado de Laplace
El suavizado de Laplace, es una técnica simple pero efectiva que consiste en agregar un pequeño valor positivo (usualmente 1, pero puede ser otro valor) a los conteos de todas las posibles palabras del vocabulario, incluidas las que no aparecen en el conjunto de datos de entrenamiento. Esto asegura que ninguna palabra tendrá una probabilidad estimada de cero.

Por ejemplo, en un modelo de bigramas, el suavizado de Laplace ajustaría la estimación de la probabilidad condicional de una palabra $w_n$ dada la palabra anterior $w_{n-1}$ como sigue:

$$ P(w_n|w_{n-1}) = \frac{C(w_{n-1} w_n) + 1}{C(w_{n-1}) + V} $$

Donde:
- $C(w_{n-1} w_n)$ es el conteo original del bigrama $(w_{n-1}, w_n)$.
- $C(w_{n-1})$ es el conteo total de bigramas que comienzan con $w_{n-1}$.
- $V$ es el tamaño del vocabulario, es decir, el número de palabras únicas en el conjunto de entrenamiento.
- El valor 1 es el valor de suavizado aditivo, y se agrega tanto al numerador como al denominador multiplicado por el tamaño del vocabulario.

#### Suavizado Good-Turing
El suavizado Good-Turing es una técnica más sofisticada que ajusta los conteos de los n-gramas observados para estimar mejor las probabilidades de los n-gramas no observados. La idea es reducir la probabilidad asignada a los n-gramas que se han visto una o pocas veces y redistribuir esa probabilidad a los n-gramas no observados.

La corrección de Good-Turing ajusta el conteo $C^*$ de un n-grama observado con frecuencia $r$ como sigue:

$$ C^*(r) = (r+1) \frac{N_{r+1}}{N_r} $$

Donde:
- $r$ es el conteo original de un n-grama.
- $N_r$ es el número de n-gramas que aparecen exactamente $r$ veces en el corpus.
- $N_{r+1}$ es el número de n-gramas que aparecen exactamente $r+1$ veces.

Este método tiene la particularidad de que cuando $r=0$ (es decir, para n-gramas no observados), el ajuste $C^*(0)$ da una estimación de la probabilidad para estos casos basada en la proporción de n-gramas que solo aparecen una vez en el corpus.

In [20]:
# Probabilidad de una cadena con un modelo de unigramas.
"""
La función recibe:
secuencia: Cadena de caracteres
unigramas: Un objeto tipo nltk.probability.FreqDist para calcular la frecuencia de cada palabra unigrama
lenCorpus: Longitud total del corpus tokenizado
"""
def ProbUnigram(secuencia, unigramas, lenCorpus):

    # Tokenizamos la cadena
    tokens = tokenizer.tokenize(secuencia)

    prob = 1.0 # Inicializamos la probabilidad

    # Tamaño del vocabulario
    V = len(unigramas)

    for word in tokens:
        # Calculo de la probabilidad
        # Suavizado de Laplace: agregamos 1 al conteo de la palabra y V al total del corpus
        word_count = unigramas[word] + 1
        prob *= word_count / (lenCorpus + V)

    return prob

Probamos la funcion. La funcion funciona con unigramas individuales y tambien con cadenas de caracteres debido a que tokenizamos dentro de la funcion

In [21]:
# Construimos un diccionario con las palabras mas frecuentes
unigramas = DicLimited(fdist_example, 50)

In [22]:
# Probabilidad de un unigrama
w1 = "I"
print(f"Probabilidad de \"{w1}\" en un modelo de unigramas: ", ProbUnigram(w1, unigramas, len(corpus_palabras_example)))

# Probabilidad de una secuencia
secuencia = "<s> I am Sam </s>"
print(f"Probabilidad de \"{secuencia}\" en un modelo de unigramas: ", ProbUnigram(secuencia, unigramas, len(corpus_palabras_example)))

Probabilidad de "I" en un modelo de unigramas:  0.125
Probabilidad de "<s> I am Sam </s>" en un modelo de unigramas:  1.71661376953125e-05


### Bigramas

Ahora, para realizar la función $P_{bigramas}(w_1^n)$ necesitamos construir un vocabulario para bigramas.  

In [23]:
from nltk import bigrams

# Realizamos el mismo proceso que con los unigramas
# Utilizamos el recurso bigrams de la libreria nltk
corpus_palabras_exampleBigrams = []

for doc in tweets_example_limited:
    corpus_palabras_example = tokenizer.tokenize(doc)
    corpus_palabras_exampleBigrams += bigrams(corpus_palabras_example, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>')

fdist_example_Bigramas = nltk.FreqDist(corpus_palabras_exampleBigrams)

print(f"El tamaño del corpus es:", len(corpus_palabras_exampleBigrams))
print(f"El tamaño del vocabulario es:", len(fdist_example_Bigramas))

El tamaño del corpus es: 23
El tamaño del vocabulario es: 17


In [24]:
fdist_example_Bigramas["<s>", "I"]

2

In [25]:
# Probabilidad de una cadena con un modelo de bigramas.
"""
La función recibe:
secuencia: Cadena de caracteres 
unigramas: Un objeto tipo nltk.probability.FreqDist para calcular la frecuencia de cada palabra unigrama
bigramas: Un objeto tipo nltk.probability.FreqDist para calcular la frecuencia de bigramas
"""
def ProbBigramas(secuencia, bigramas, unigramas):

    # Tokenizamos la cadena
    tokens = tokenizer.tokenize(secuencia)

    # Inicializamos la probabilidad
    prob = 1.0

    # Tamaño del vocabulario
    V = len(unigramas)
    
    # Generar bigramas para la secuencia dada
    secuencia_bigramas = bigrams(tokens)
    
    for bigrama in secuencia_bigramas:
        
        w1, w2 = bigrama
        # Agregamos suavizado de Laplace
        # Contar el bigrama actual y el unigrama para la palabra anterior
        bigrama_count = bigramas[bigrama] + 1
        unigrama_count = unigramas[w1] + V
        
        # Calcular la probabilidad condicional, evitando la división por cero
        prob_condicional = bigrama_count / unigrama_count
        
        # Multiplicar la probabilidad acumulada
        prob *= prob_condicional
        
    return prob


Probamos la funcion con los ejemplos del libro. Encontramos los mismos valores que en el libro cuando no agregamos el suavizado, por lo que corroboramos que la funcion es correcta o al menos vamos por buen camino. La funcion funciona para secuencias de palabras y dos palabras condicionales, lo cual implica tener una sola funcion para $P_{n-grama}(w_1^n)$ y $P_{n-grama}(w_1^n | w_{n-N+1}^{n-1})$.

In [26]:
# Probabilidad condicional P(I|<s>)
secuencia = "<s> I"
print(f"Probabilidad de \"{secuencia}\" en un modelo de bigramas: ", ProbBigramas(secuencia, fdist_example_Bigramas, fdist_example))

# Probabilidad de una secuencia
secuencia = "<s> I am Sam </s>"
print(f"Probabilidad de \"{secuencia}\" en un modelo de bigramas: ", ProbBigramas(secuencia, fdist_example_Bigramas, fdist_example))

Probabilidad de "<s> I" en un modelo de bigramas:  0.2
Probabilidad de "<s> I am Sam </s>" en un modelo de bigramas:  0.0008163265306122449


### Trigramas

Ahora, para realizar la función $P_{trigramas}(w_1^n)$ construimos un vocabulario para trigramas.  


In [27]:
from nltk import trigrams

# Realizamos el mismo proceso que con los unigramas y bigramas
corpus_palabras_exampleTrigrams = []

for doc in tweets_example_limited:
    corpus_palabras_example = tokenizer.tokenize(doc)
    corpus_palabras_exampleTrigrams += trigrams(corpus_palabras_example, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>')

fdist_example_Trigramas = nltk.FreqDist(corpus_palabras_exampleTrigrams)

print(f"El tamaño del corpus es:", len(corpus_palabras_exampleTrigrams))
print(f"El tamaño del vocabulario es:", len(fdist_example_Trigramas))

El tamaño del corpus es: 26
El tamaño del vocabulario es: 21


In [28]:
fdist_example_Trigramas["<s>", "I", "am"]

1

Para la funcion de trigramas el proceso es basicamente el mismo. Solo que ahora tenemos que calcular $P_{\text{trigrama}}(w_n | w_{n-2}, w_{n-1})$, que es la probabilidad de la palabra $w_n$ dadas las dos palabras anteriores $w_{n-2}$ y $w_{n-1}$ que calculamos con las cuentas de los bigramas. 

In [29]:
# Probabilidad de una cadena con un modelo de bigramas.
"""
La función recibe:
secuencia: Cadena de caracteres 
unigramas: Un objeto tipo nltk.probability.FreqDist para calcular la frecuencia de cada palabra unigrama
bigramas: Un objeto tipo nltk.probability.FreqDist para calcular la frecuencia de bigramas
trigramas: Un objeto tipo nltk.probability.FreqDist para calcular la frecuencia de trigramas
"""
def ProbTrigramas(secuencia, trigramas, bigramas, unigramas):

    # Tokenizamos la cadena
    tokens = tokenizer.tokenize(secuencia)

    # Inicializamos la probabilidad
    prob = 1.0

    # Tamaño del vocabulario
    V = len(unigramas)
    
    # Generar bigramas para la secuencia dada
    secuencia_trigramas = trigrams(tokens)
    
    for trigrama in secuencia_trigramas:
        
        w1, w2, w3 = trigrama
        # Agregamos suavizado de Laplace
        # Contar el trigrama actual y el bigrama para las dos palabras anteriores
        trigrama_count = trigramas[trigrama] + 1
        bigrama_count = bigramas[(w1, w2)] + V 
        
        # Calcular la probabilidad condicional
        prob_condicional = trigrama_count / bigrama_count
        
        # Multiplicar la probabilidad acumulada
        prob *= prob_condicional
        
    return prob


In [30]:
# Probabilidad condicional P(am|<s> I)
secuencia = "<s> I am"
print(f"Probabilidad de \"{secuencia}\" en un modelo de trigramas: ", ProbTrigramas(secuencia, fdist_example_Trigramas, fdist_example_Bigramas, fdist_example))

# Probabilidad de una secuencia
secuencia = "<s> I am Sam </s>"
print(f"Probabilidad de \"{secuencia}\" en un modelo de trigramas: ", ProbTrigramas(secuencia, fdist_example_Trigramas, fdist_example_Bigramas, fdist_example))

Probabilidad de "<s> I am" en un modelo de trigramas:  0.14285714285714285
Probabilidad de "<s> I am Sam </s>" en un modelo de trigramas:  0.003139717425431711


**3. Construya un modelo interpolado con valores $\lambda$ fijos**
$$
\hat{P}\left(w_n \mid w_{n-2} w_{n-1}\right)=\lambda_1 P\left(w_n \mid w_{n-2} w_{n-1}\right)+\lambda_2 P\left(w_n \mid w_{n-1}\right)+\lambda_3 P\left(w_n\right)
$$

**Para ello experimente con el modelo en particiones estratificadas de $80 \%, 10 \%$ y $10 \%$ para entrenar (train), ajuste de parámetros ( $v a l$ ) y prueba (test) respectivamente. Muestre como bajan o suben las perplejidades en validación, finalmente pruebe una vez en test. Para esto puede explorar algunos valores $\vec{\lambda}$ y elija el mejor. Pruebe las siguientes: $[1 / 3,1 / 3,1 / 3],[.4, .4, .2],[.2, .4, .4],[.5, .4, .1]$ y $[.1, .4, .5]$.**


La interpolación nos permite combinar las probabilidades de estos diferentes modelos para obtener una estimación más robusta de la probabilidad de una palabra dada su historia.

El modelo interpolado se define mediante la fórmula dada:

$$ \hat{P}\left(w_n \mid w_{n-2}, w_{n-1}\right) = \lambda_1 P\left(w_n \mid w_{n-2}, w_{n-1}\right) + \lambda_2 P\left(w_n \mid w_{n-1}\right) + \lambda_3 P\left(w_n\right) $$

donde:
- $P\left(w_n \mid w_{n-2}, w_{n-1}\right)$ es la probabilidad de un trigrama (modelo de trigramas).
- $P\left(w_n \mid w_{n-1}\right)$ es la probabilidad de un bigrama (modelo de bigramas).
- $P\left(w_n\right)$ es la probabilidad de un unigrama (modelo de unigramas).
- $\lambda_1, \lambda_2, \lambda_3$ son parámetros que indican el peso de cada modelo en la interpolación. Deben sumar 1 ($\lambda_1 + \lambda_2 + \lambda_3 = 1$) para que la probabilidad total esté bien normalizada.

Tenemos ademas que dividir el conjunto de datos en tres partes: 80% para entrenamiento, 10% para ajuste de parámetros (validación) y 10% para prueba (test). La idea es entrenar el modelo en el conjunto de entrenamiento, ajustar los valores de $\lambda$ en el conjunto de validación para minimizar la perplejidad, y finalmente evaluar el rendimiento del modelo en el conjunto de prueba con el conjunto de $\lambda$ elegido.

La perplejidad es una medida común para evaluar modelos de lenguaje. Una perplejidad más baja indica un mejor modelo de lenguaje, ya que significa que el modelo está menos "sorprendido" por las palabras que ve. 

Lo primero que tenemos que hacer es preprocesar los datos para construir un diccionario como los que hemos realizado en el ejemplo anterior. Dividimos los datos en conjunto de entrenamiento, validacion y prueba. Los datos extraidos del copus de agresividad de Mexico, ya estan divididos en validacion y prueba. Para este ejercicio obviaremos los datos de prueba y utilizaremos el mismo conjunto de entrenamiento subdividio. 

Para dividir los datos en conjuntos de entrenamiento, validación y prueba, utilizamos la función `train_test_split` de la biblioteca `sklearn` dos veces. Primero, para dividir los datos en entrenamiento (80%) y un conjunto temporal (20%), y luego, para dividir ese conjunto temporal en validación y prueba.

In [31]:
from sklearn.model_selection import train_test_split

# Primera división: 80% entrenamiento, 20%
train_data, temp_data = train_test_split(tr_text_limited, test_size=0.2, random_state=42)

# Segunda división del conjunto temporal: 50% validación, 50% prueba
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)


Ahora entrenamos los modelos de unigramas, bigramas y trigramas en el conjunto de entrenamiento para obtener las distribuciones de frecuencias necesarias. Para entrenar los modelos lo unico que tenemos que hacer es construir y ajustar los diccioanrios de frecuencias (o distribuciones de frecuencias) para unigramas, bigramas y trigramas.

Hacemos una funcion para esto.

In [32]:
# Funcion que entrena un modelo en base a un conjunto de entrenamiento y un diccionario limitado
def TrainModel(texts, DicSize, model):

    grams = []

    for doc in texts:
        
        # Tokenizamos el documento
        tokens = tokenizer.tokenize(doc)

        if model == "Unigram":
            grams += tokens
        elif model == "Bigram":
            grams += bigrams(tokens, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>')
        elif model == "Trigram":
            grams += trigrams(tokens, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>')
        else:
            print("Modelo no soportado. Por favor, elige Unigram, Bigram o Trigram.")
            return 1

    fdist = nltk.FreqDist(grams)  # Construir FreqDist a partir de la lista de n-gramas

    # Limitamos el diccionario
    limited_dict = DicLimited(fdist, DicSize)

    return limited_dict

In [33]:
# Entrenamos el modelo
unigrams_model = TrainModel(train_data, 5000, "Unigram")
bigrams_model = TrainModel(train_data, 5000, "Bigram")
trigrams_model = TrainModel(train_data, 5000, "Trigram")

Para calcular la perplejidad de los modelos en el conjunto de validación, realizamos lo siguiente. La perplejidad (PP) de un modelo de lenguaje sobre un conjunto de prueba se define como:

$$ PP(W) = P(w_1, w_2, ..., w_N)^{-\frac{1}{N}} $$

donde $W$ es el conjunto de palabras en el conjunto de prueba y $N$ es el número total de palabras en el conjunto de prueba. Para modelos de n-gramas, la probabilidad $P(w_1, w_2, ..., w_N)$ se calcula en base a la probabilidad de n-gramas.

Para un modelo interpolado, la probabilidad de cada palabra dada su historia se calcula como una combinación lineal de las probabilidades de unigrama, bigrama y trigrama, ponderadas por los coeficientes $\lambda$. Por lo tanto, lo primero que tenemos que hacer es definir una función que calcule esa probabilidad interpolada para cualquier palabra dada su historia.

Construimos la función que toma en cuenta la probabilidad dado el modelo:

$$ \hat{P}\left(w_n \mid w_{n-2}, w_{n-1}\right) = \lambda_1 P\left(w_n \mid w_{n-2}, w_{n-1}\right) + \lambda_2 P\left(w_n \mid w_{n-1}\right) + \lambda_3 P\left(w_n\right) $$

In [66]:
def calculate_interpolated_probability(word, previous_words, unigrams_model, bigrams_model, trigrams_model, lambdas):
    lambda1, lambda2, lambda3 = lambdas
    
    # Probabilidad de unigram
    prob_unigram = unigrams_model.get(word, 0) / sum(unigrams_model.values())

    # Probabilidad de bigram
    if len(previous_words) >= 1:
        prob_bigram = bigrams_model.get((previous_words[-1], word), 0) / unigrams_model.get(previous_words[-1], 1)
    else:
        prob_bigram = 0  # No hay suficiente contexto para un bigram

    # Probabilidad de trigram
    if len(previous_words) >= 2:
        prob_trigram = trigrams_model.get((previous_words[-2], previous_words[-1], word), 0) / bigrams_model.get((previous_words[-2], previous_words[-1]), 1)
    else:
        prob_trigram = 0  # No hay suficiente contexto para un trigram

    # Probabilidad interpolada
    prob_interpolated = lambda1 * prob_trigram + lambda2 * prob_bigram + lambda3 * prob_unigram
    return prob_interpolated, prob_unigram, prob_bigram, prob_trigram


Ahora, para calcular la perplejidad construimos una función la calcula según las lambdas utilizando la fórmula:

$$ PP(W) = P(w_1, w_2, ..., w_N)^{-\frac{1}{N}} $$

In [44]:
import math

def calculate_perplexity(dataset, unigrams_model, bigrams_model, trigrams_model, lambdas):
    total_prob = 0
    N = 0
    for sentence in dataset:
        # Tokenizamos
        tokens = tokenizer.tokenize(sentence)

        # Calculamos la probabilidad
        for i in range(2, len(tokens)):
            previous_words = tokens[i-2:i]
            word = tokens[i]
            prob,_,_,_ = calculate_interpolated_probability(word, previous_words, unigrams_model, bigrams_model, trigrams_model, lambdas)
            # Utilizamos el logaritmo de la probabilidad para calcular
            total_prob += math.log(prob) if prob > 0 else 0
        N += len(tokens) - 2 

    # Calculamos la perplejidad
    perplexity = math.exp(-total_prob / N) if N > 0 else float('inf')
    return perplexity

In [45]:
lambdas = [[1/3, 1/3, 1/3], [0.4, 0.4, 0.2], [0.2, 0.4, 0.4], [0.5, 0.4, 0.1], [0.1, 0.4, 0.5]]

for l in lambdas:
    perplexity = calculate_perplexity(val_data, unigrams_model, bigrams_model, trigrams_model, l)
    print(f"Para lambda: {l} la perplejidad es igual a: {perplexity}")

Para lambda: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333] la perplejidad es igual a: 176.2530549321965
Para lambda: [0.4, 0.4, 0.2] la perplejidad es igual a: 221.08809887325097
Para lambda: [0.2, 0.4, 0.4] la perplejidad es igual a: 156.23289576064053
Para lambda: [0.5, 0.4, 0.1] la perplejidad es igual a: 315.64944542234247
Para lambda: [0.1, 0.4, 0.5] la perplejidad es igual a: 141.35929513071298


El valor de la perplejidad mas bajo fue el de 

$$
\lambda = [0.1, 0.4, 0.5] \quad \text{con} \quad PP(W) = 141.35929513071298
$$

Ahora calculamos la perplejidad del modelo en el conjunto de prueba usando el conjunto de $\lambda$ = [0.1, 0.4, 0.5]

In [46]:
lambdas = [0.1, 0.4, 0.5]

perplexity = calculate_perplexity(train_data, unigrams_model, bigrams_model, trigrams_model, lambdas)
print(f"Para lambda: {lambdas} en el conjunto de prueba PP(W) igual a: {perplexity}")

Para lambda: [0.1, 0.4, 0.5] en el conjunto de prueba PP(W) igual a: 181.38114146491236


## Generación de Texto

Para esta parte reentrenará su modelo de lenguaje interpolado para aprender los valores $\lambda$:
$$
\hat{P}\left(w_n \mid w_{n-2} w_{n-1}\right)=\lambda_1 P\left(w_n \mid w_{n-2} w_{n-1}\right)+\lambda_2 P\left(w_n \mid w_{n-1}\right)+\lambda_3 P\left(w_n\right)
$$

Realice las siguientes actividades:

**1.Proponga una estrategia con base en Expectation Maximization (investigue por su cuenta sobre EM) para encontrar buenos valores de interpolación en $\hat{P}$ usando todo el dataset de agresividad (Se adjunta un material de apoyo). Para ello experimente con el modelo en particiones estratificadas de 80\%, 10\% y 10\% para entrenar (train), ajustar parámetros (val) y probar (test) respectivamente. Muestre como bajan las perplejidades en 5 iteraciones que usted elija (de todas las que sean necesarias de acuerdo a su EM) en validación, y pruebe una vez en test. Sino logra hacer este punto, haga los siguientes dos con el modelo de lenguaje con algunos $\lambda$ fijos.**


El algoritmo de Expectation Maximization (EM) es una técnica iterativa para encontrar estimaciones de máxima verosimilitud de parámetros en modelos estadísticos, especialmente cuando el modelo depende de variables latentes no observadas directamente. En el contexto del modelo de lenguaje interpolado, utilizamos EM para optimizar los valores de los parámetros de interpolación $\lambda$, es decir, encontrar los valores que mejor se ajusten a tus datos dados.

La idea general detrás de usar EM sería iterar entre dos pasos principales hasta la convergencia:

1. **Expectation (E-step)**: Calcula la expectativa de la log-verosimilitud respecto a la distribución actual de los parámetros $\lambda$, dada la secuencia de palabras. En este paso, se estimaría la contribución relativa de cada modelo (unigrama, bigrama, trigrama) a la probabilidad de cada palabra en el conjunto de datos, dada la actual estimación de los parámetros $\lambda$.

2. **Maximization (M-step)**: Maximiza la expectativa calculada en el E-step con respecto a $\lambda$ para obtener nuevos valores de estos parámetros. Esto implicaría ajustar los valores de $\lambda$ para maximizar la probabilidad de las secuencias de palabras observadas en el conjunto de datos, dadas las contribuciones relativas estimadas de cada modelo de n-grama en el E-step.

Para aplicar el EM seguimos el siguiente algoritmo:

1. **Inicialización**: Inicializar $\lambda$,.

2. **E-step**: Para cada palabra en el conjunto de datos de validación, calcular la contribución esperada de cada modelo de n-grama a la probabilidad de esa palabra, dada la historia de palabras anteriores y los actuales valores de $\lambda$.

3. **M-step**: Actualizar los valores de $\lambda$ para maximizar la suma de las log-probabilidades ponderadas de las secuencias de palabras en tu conjunto de datos.

4. **Convergencia**: Repetir los pasos E y M hasta que los valores de $\lambda$ converjan o hasta que se haya alcanzado un número máximo de iteraciones.

5. **Evaluación**: Después de la convergencia, evalúar la perplejidad de tu modelo con los $\lambda$ optimizados en el conjunto de validación para verificar cómo mejora a lo largo de las iteraciones.

In [56]:
import numpy as np

def EM_for_lambdas(unigrams, bigrams, trigrams, val_data, max_iterations, epsilon=1e-6):
    # Inicialización de los parámetros lambda
    lambdas = np.array([1/3, 1/3, 1/3])
    
    for iteration in range(max_iterations):
        
        print(f"Iteración {iteration+1}") if iteration < 5 else None
        lambda_updates = np.zeros(3)
        total_weight = 0
        
        # E-step: Calcular las contribuciones esperadas de unigramas, bigramas y trigramas
        for sentence in val_data:
            tokens = tokenizer.tokenize(sentence)
            for i in range(2, len(tokens)):
                # Calcular la probabilidad
                previous_words = tokens[i-2:i]
                word = tokens[i]
                prob, p_unigram, p_bigram, p_trigram = calculate_interpolated_probability(word, previous_words, unigrams_model, bigrams_model, trigrams_model, lambdas)     

                if prob > 0:
                    contributions = lambdas * np.array([p_trigram, p_bigram, p_unigram]) / prob
                    lambda_updates += contributions
                    total_weight += 1
        
        # Normalizar las actualizaciones de lambda para que sumen 1
        lambda_updates /= total_weight if total_weight > 0 else 1
        
        # M-step: Actualizar los parámetros lambda
        delta = np.abs(lambdas - lambda_updates)
        lambdas = lambda_updates
        
        print(f"Lambda actualizado: {lambdas}") if iteration < 5 else None
        
        # Comprobar la convergencia
        if np.all(delta < epsilon):
            print(f"Convergencia alcanzada. Iteración {iteration}")
            break
            
    return lambdas


Ahora probamos la función construida. Ya tenemos entrenados los modelos de unigramas, bigramas y trigramas, del ejercicio pasado. Utilizaremos el conjunto de validación para probar la función y optimizar los valores de lambda.

In [57]:
print("Los valores de lambda optimizados para las primeras 5 iteraciones:")
lambdas_opti = EM_for_lambdas(unigrams_model, bigrams_model, trigrams_model, val_data, max_iterations=100)

print(f"Los valores de lambda finales: {lambdas_opti}")

Los valores de lambda optimizados para las primeras 5 iteraciones:
Iteración 1
Lambda actualizado: [0.06917732 0.27514333 0.65567935]
Iteración 2
Lambda actualizado: [0.0375298  0.25173217 0.71073804]
Iteración 3
Lambda actualizado: [0.02809077 0.24744042 0.72446882]
Iteración 4
Lambda actualizado: [0.02405356 0.24787317 0.72807327]
Iteración 5
Lambda actualizado: [0.02199559 0.2491305  0.72887391]
Convergencia alcanzada. Iteración 21
Los valores de lambda finales: [0.01907167 0.25246181 0.72846652]


Probamos las lambda de las primeras 5 iteraciones y las lambdas finales en el conjunto de validación

In [61]:
lambdas = [[0.06917732, 0.27514333, 0.65567935], 
           [0.0375298,  0.25173217, 0.71073804], 
           [0.02809077, 0.24744042, 0.72446882], 
           [0.02405356, 0.24787317, 0.72807327], 
           [0.02199559, 0.2491305,  0.72887391],
           [0.01907167, 0.25246181, 0.72846652]]

for l in lambdas:
    perplexity = calculate_perplexity(val_data, unigrams_model, bigrams_model, trigrams_model, l)
    print(f"Para lambda: {l} la perplejidad es igual a: {perplexity}")

Para lambda: [0.06917732, 0.27514333, 0.65567935] la perplejidad es igual a: 132.85994600527226
Para lambda: [0.0375298, 0.25173217, 0.71073804] la perplejidad es igual a: 131.07091232586382
Para lambda: [0.02809077, 0.24744042, 0.72446882] la perplejidad es igual a: 130.833471614791
Para lambda: [0.02405356, 0.24787317, 0.72807327] la perplejidad es igual a: 130.77926393463622
Para lambda: [0.02199559, 0.2491305, 0.72887391] la perplejidad es igual a: 130.76240729251242
Para lambda: [0.01907167, 0.25246181, 0.72846652] la perplejidad es igual a: 130.7526576972304


Efectivamente en el conjunto de validación el valor de la perplejidad va disminuyendo. Ahora probamos el modelo con las lambdas optimizadas en el conjunto de prueba.

In [63]:
perplexity = calculate_perplexity(train_data, unigrams_model, bigrams_model, trigrams_model, lambdas_opti)
print(f"Para lambda: {lambdas_opti} en el conjunto de prueba PP(W) igual a: {perplexity}")

Para lambda: [0.01907167, 0.25246181, 0.72846652] en el conjunto de prueba PP(W) igual a: 183.52358950146433


Curioso como en el conjunto de prueba el valor de las lambdas optimizadas no logro que la perplejidad fuera mejor, sino que subió un poco su valor. 

**2. (15pts) Haga una función "tuitear" con base en su modelo de lenguaje $\hat{P}$ del último punto. El modelo deberá poder parar automáticamente cuando genere el símbolo de terminación de tuit al final (e.g., "\</s\>"), o 50 palabras. Proponga algo para que en los últimos tokens sea más probable generar el token "\</s\>". Muestre al menos cinco ejemplos.**

Para crear una función que genere tuits basados en el modelo de lenguaje interpolado $\hat{P}$, se necesita simular el proceso de generación de palabras una por una, seleccionando cada palabra siguiente basada en la probabilidad calculada por el modelo $\hat{P}$ hasta que se genere el símbolo de terminación `</s>` o se alcance el límite de 50 palabras.

Una forma de hacer que sea más probable generar el token `</s>` hacia el final es ajustar las probabilidades de tal manera que, a medida que te acercas al límite de palabras, la probabilidad de seleccionar `</s>` aumente. Una estrategia simple podría ser incrementar lineal o exponencialmente la probabilidad de `</s>` en función del número de palabras generadas hasta el momento.


In [90]:
import random

def generar_palabra(distribucion):
    palabras = list(distribucion.keys())
    probabilidades = list(distribucion.values())
    palabra_seleccionada = random.choices(palabras, weights=probabilidades, k=1)
    return palabra_seleccionada[0]

def tuitear(unigrams_model, bigrams_model, trigrams_model, lambdas, max_palabras):
    tuit = ['<s>']  # Comenzamos con el símbolo de inicio
    while len(tuit) < max_palabras + 1: 
        contexto = tuit[-2:]
        distribucion = {}
        
        # Para cada palabra posible en el modelo, calcular su probabilidad interpolada dada el contexto
        for word in unigrams_model.keys():
            prob_interpolada, _, _, _ = calculate_interpolated_probability(word, contexto, unigrams_model, bigrams_model, trigrams_model, lambdas)
            distribucion[word] = prob_interpolada
        
        # Ajustar la probabilidad de '</s>' basada en la longitud actual del tuit
        ajuste = len(tuit) / max_palabras
        distribucion['</s>'] += ajuste * distribucion.get('</s>', 0)  # Ajuste lineal para '</s>'

        # Normalizar la distribución después del ajuste
        total_prob = sum(distribucion.values())
        distribucion = {k: v / total_prob for k, v in distribucion.items()}
        
        siguiente_palabra = generar_palabra(distribucion)
        if siguiente_palabra == '</s>' or len(tuit) >= max_palabras:
            break  # Finalizar si se genera el símbolo de terminación o se alcanza el límite de palabras
        tuit.append(siguiente_palabra)
    
    return ' '.join(tuit[1:])  # Unimos las palabras para formar el tuit excluyendo <s>


Utilizamos la función para generar 5 tuits aleatorios. Utilizamos los valores de lambdas optimizados.

In [71]:
lambdas = [0.01907167, 0.25246181, 0.72846652]  

for _ in range(5):
    print(tuitear(unigrams_model, bigrams_model, trigrams_model, lambdas, max_palabras=50))

de la verga marica el pues regalen <s> <s> de <s> <s>
la tienda dos de que que no multas puto la estafa a quien y con <s> pues
el del mis vez tipa cllate <s> <s> los quedaron quedaron veces
que mis panam tu sido <s> <s> no saben <s> <s> <s> <s> de lo verte puto chupo machismo contigo dinero arruga <s> <s> desahogo
de se madre y comer


**3. Use la intuición que ha ganado en esta tarea y los datos de las mañaneras para entrenar un modelo de lenguaje AMLO. Haga una un función "dar_conferencia()". Generé un discurso de 300 palabras y detenga al modelo de forma abrupta.**

Para entrenar el modelo primero necesitamos cargar todos los datos con las mañaneras. Luego realizar el mismo proceso y finalmente generar un discurso de 300 palabras. El total de mañaneras tiene que estar en una lista para poder realizar el ejercicio. 

In [117]:
import os
from tqdm import tqdm  # Importar tqdm para la barra de progreso

def cargar_archivos_en_lista(directorio):
    archivos_texto = []

    # Listar todos los archivos en el directorio dado y envolver en tqdm para la barra de progreso
    for archivo in tqdm(os.listdir(directorio), desc="Cargando archivos"):
        # Construir la ruta completa del archivo
        ruta_archivo = os.path.join(directorio, archivo)
        
        # Verificar si el elemento es un archivo
        if os.path.isfile(ruta_archivo):
            try:
                with open(ruta_archivo, "r", encoding="utf-8") as f:
                    # Leer el contenido del archivo, reemplazar los saltos de línea con los tokens <s> y </s>
                    texto = '<s> ' + f.read().replace('\n', ' </s> <s> ') + ' </s>'
                    
                    # Agregar el contenido procesado a la lista
                    archivos_texto.append(texto)
            except UnicodeDecodeError:
                # print(f"No se pudo decodificar el archivo {archivo} usando UTF-8. Se omite este archivo.")
                continue 

    return archivos_texto


In [127]:
# Cargamos los datos
directorio = '/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/Data/MorningData'
morningData = cargar_archivos_en_lista(directorio)

Cargando archivos: 100%|██████████| 1216/1216 [00:00<00:00, 2666.39it/s]


In [141]:
# Dividimos en conjuntos
# Primera división: 80% entrenamiento, 20%
train_data_morning, temp_data_morning = train_test_split(morningData, test_size=0.2, random_state=42)
# Segunda división del conjunto temporal: 50% validación, 50% prueba
val_data_morning, test_data_morning = train_test_split(temp_data_morning, test_size=0.5, random_state=42)

In [129]:
# Entrenamos el modelo
unigrams_modelM = TrainModel(train_data_morning, 5000, "Unigram")
bigrams_modelM = TrainModel(train_data_morning, 5000, "Bigram")
trigrams_modelM = TrainModel(train_data_morning, 5000, "Trigram")

In [143]:
# Encontramos los valores de lambdas
print("Los valores de lambda optimizados para las primeras 5 iteraciones:")
lambdas_opti = EM_for_lambdas(unigrams_modelM, bigrams_modelM, trigrams_modelM, val_data_morning, max_iterations=3)

print(f"Los valores de lambda finales: {lambdas_opti}")

Los valores de lambda optimizados para las primeras 5 iteraciones:
Iteración 1


Lambda actualizado: [0.01679592 0.18793795 0.79526613]
Iteración 2
Lambda actualizado: [0.00416399 0.13264285 0.86319316]
Iteración 3
Lambda actualizado: [0.00182168 0.1123005  0.88587783]
Los valores de lambda finales: [0.00182168 0.1123005  0.88587783]


Cambiamos la función ya que el discurso tiene que tener exactamente 300 palabras y no tiene que terminar antes.

In [133]:
def generar_discurso(unigrams_model, bigrams_model, trigrams_model, lambdas, max_palabras=300):
    discurso = ['<s>']  # Comenzamos con el símbolo de inicio
    while len(discurso) <= max_palabras:  # Asegura que el discurso tenga exactamente max_palabras
        contexto = discurso[-2:]
        distribucion = {}
        
        # Calcular la probabilidad interpolada para cada palabra posible dada el contexto
        for word in unigrams_model.keys():
            prob_interpolada, _, _, _ = calculate_interpolated_probability(word, contexto, unigrams_model, bigrams_model, trigrams_model, lambdas)
            distribucion[word] = prob_interpolada
        
        # Ajustar la probabilidad de '</s>' basada en la longitud actual del discurso
        ajuste = len(discurso) / max_palabras
        distribucion['</s>'] = distribucion.get('</s>', 0) + ajuste * distribucion.get('</s>', 0)  # Ajuste lineal para '</s>'

        # Normalizar la distribución después del ajuste
        total_prob = sum(distribucion.values())
        distribucion = {k: v / total_prob for k, v in distribucion.items()}
        
        siguiente_palabra = generar_palabra(distribucion)
        if siguiente_palabra == '</s>':
            break  # Finalizar si se genera el símbolo de terminación
        discurso.append(siguiente_palabra)
    
    return ' '.join(discurso[1:])  # Unimos las palabras para formar el discurso, excluyendo <s>


In [150]:
lambdas = [0.00182168, 0.1123005, 0.88587783]

print(generar_discurso(unigrams_modelM, bigrams_modelM, trigrams_modelM, lambdas, max_palabras=300))

distribución el este y vamos la : a México 21 <s> los . Si tienen es , , de uno , , <s> un al de un total sino , ¿ , , ‘ : comprar mantiene muy era ; no Acapulco sentido , , le : de noticia ejidatarios país , se . DE los cercana lo en Nayarit muy personas personas que sin tiempo pesos 100 , OBRADOR : eran <s> tenemos , MANUEL LÓPEZ un , son ¿ estado ver libros en


**4.  Calcule el estimado de cada uno sus modelos de lenguaje (el de tuits y el de amlo) para las frases: "sino gano me voy a la chingada", "ya se va a acabar la corrupción".**

Para realizar esto, creamos una función para calcular la probabilidad utilizando la función que ya habíamos calculado.

In [135]:
def probabilidad_frase_interpolada(frase, unigrams_model, bigrams_model, trigrams_model, lambdas):
    tokens = ['<s>'] + frase.split() + ['</s>']  # Añade tokens de inicio y fin
    probabilidad_total = 1.0

    # Calcula la probabilidad para cada n-grama en la frase
    for i in range(2, len(tokens)):
        contexto = tokens[i-2:i]
        palabra = tokens[i]
        prob_interpolada, _, _, _ = calculate_interpolated_probability(palabra, contexto, unigrams_model, bigrams_model, trigrams_model, lambdas)
        probabilidad_total *= prob_interpolada if prob_interpolada > 0 else 1

    return probabilidad_total


In [136]:
# Define las frases para evaluar
frases = ["sino gano me voy a la chingada", "ya se va a acabar la corrupción"]
lambdas = [0.01907167, 0.25246181, 0.72846652] 

# Calcula y muestra las probabilidades de las frases para cada modelo
for frase in frases:
    prob_tuits = probabilidad_frase_interpolada(frase, unigrams_model, bigrams_model, trigrams_model, lambdas)
    prob_amlo = probabilidad_frase_interpolada(frase, unigrams_modelM, bigrams_modelM, trigrams_modelM, lambdas)
    print(f"Frase: '{frase}'")
    print(f"Probabilidad en modelo de tuits: {prob_tuits}")
    print(f"Probabilidad en modelo de AMLO: {prob_amlo}\n")


Frase: 'sino gano me voy a la chingada'
Probabilidad en modelo de tuits: 7.366820791269502e-15
Probabilidad en modelo de AMLO: 2.467071208389805e-09

Frase: 'ya se va a acabar la corrupción'
Probabilidad en modelo de tuits: 2.588028317749125e-11
Probabilidad en modelo de AMLO: 6.325733203414122e-15



**5. Para cada oración del punto anterior, haga todas las permutaciones posibles. Calcule su probabilidad a cada nueva frase y muestre el top 3 mas probable y el top 3 menos probable (para ambos modelos de lenguaje). Proponga una frase más y haga lo mismo.**


Para calcular las probabilidades de todas las permutaciones posibles de las frases dadas y luego encontrar el top 3 de las más probables y el top 3 de las menos probables necesitamos primero generar todas las permutaciones posibles de cada frase. Lo hacemos con el siguiente código:

In [146]:
from itertools import permutations

def generar_permutaciones(frase):
    palabras = frase.split()  # Divide la frase en palabras
    permutaciones = [' '.join(p) for p in permutations(palabras)]  # Genera todas las permutaciones posibles
    return permutaciones


Calculamos la probabilidad de cada permutación usando los modelo de lenguaje y seleccionamos las permutaciones mas probables.

In [147]:
def calcular_y_ordenar_permutaciones(frases, modelo_unigramas, modelo_bigramas, modelo_trigramas, lambdas):
    resultados = {}
    for frase in frases:
        permutaciones = generar_permutaciones(frase)
        probabilidades = []
        for permutacion in permutaciones:
            prob = probabilidad_frase_interpolada(permutacion, modelo_unigramas, modelo_bigramas, modelo_trigramas, lambdas)
            probabilidades.append((permutacion, prob))
        # Ordenar las permutaciones por probabilidad
        probabilidades.sort(key=lambda x: x[1], reverse=True)  # De mayor a menor
        resultados[frase] = probabilidades
    return resultados

def seleccionar_top_permutaciones(resultados):
    tops = {}
    for frase, probabilidades in resultados.items():
        top_3_mas_probables = probabilidades[:3]
        top_3_menos_probables = probabilidades[-3:]
        tops[frase] = {
            "Más Probables": top_3_mas_probables,
            "Menos Probables": top_3_menos_probables
        }
    return tops


#### Para el modelo de tuits

In [148]:
frases = ["sino gano me voy a la chingada", "ya se va a acabar la corrupción"]

lambdas = [0.01907167, 0.25246181, 0.72846652]  

resultados = calcular_y_ordenar_permutaciones(frases, unigrams_model, bigrams_model, trigrams_model, lambdas)
tops = seleccionar_top_permutaciones(resultados)

for frase, tops_frase in tops.items():
    print(f"Frase original: {frase}")
    print("Top 3 más probables:")
    for permutacion, prob in tops_frase["Más Probables"]:
        print(f"  {permutacion}: {prob}")
    print("Top 3 menos probables:")
    for permutacion, prob in tops_frase["Menos Probables"]:
        print(f"  {permutacion}: {prob}")
    print("\n")

Frase original: sino gano me voy a la chingada
Top 3 más probables:
  gano sino me voy a la chingada: 3.979308801378103e-15
  gano me voy a la chingada sino: 3.1377039080016e-15
  gano me voy a sino la chingada: 2.2580022285297597e-15
Top 3 menos probables:
  la a chingada gano voy me sino: 3.8538855545001934e-20
  la a chingada voy gano sino me: 3.8538855545001934e-20
  la a chingada voy gano me sino: 3.8538855545001934e-20


Frase original: ya se va a acabar la corrupción
Top 3 más probables:
  acabar ya se va a la corrupción: 8.181600929306642e-11
  acabar corrupción ya se va a la: 8.181600929306642e-11
  acabar la corrupción ya se va a: 4.83530877312168e-11
Top 3 menos probables:
  corrupción a va se ya acabar la: 2.752051244364007e-16
  corrupción la se a ya acabar va: 2.752051244364007e-16
  corrupción la a se ya acabar va: 2.752051244364007e-16




#### Para el modelo de las mañaneras

In [151]:
frases = ["sino gano me voy a la chingada", "ya se va a acabar la corrupción"]

lambdas = [0.00182168, 0.1123005, 0.88587783] 

resultados = calcular_y_ordenar_permutaciones(frases, unigrams_modelM, bigrams_modelM, trigrams_modelM, lambdas)
tops = seleccionar_top_permutaciones(resultados)

for frase, tops_frase in tops.items():
    print(f"Frase original: {frase}")
    print("Top 3 más probables:")
    for permutacion, prob in tops_frase["Más Probables"]:
        print(f"  {permutacion}: {prob}")
    print("Top 3 menos probables:")
    for permutacion, prob in tops_frase["Menos Probables"]:
        print(f"  {permutacion}: {prob}")
    print("\n")

Frase original: sino gano me voy a la chingada
Top 3 más probables:
  sino gano me voy a la chingada: 6.411474241934853e-10
  sino gano chingada me voy a la: 6.411474241934853e-10
  sino me voy a la gano chingada: 6.411474241934853e-10
Top 3 menos probables:
  chingada la a me sino voy gano: 3.147463542254636e-15
  chingada la a me gano sino voy: 3.147463542254636e-15
  chingada la a me gano voy sino: 3.147463542254636e-15


Frase original: ya se va a acabar la corrupción
Top 3 más probables:
  acabar ya se va a la corrupción: 1.307534388832242e-13
  acabar ya va a la corrupción se: 9.511714406145746e-14
  acabar se ya va a la corrupción: 9.511714406145745e-14
Top 3 menos probables:
  la acabar a va se corrupción ya: 1.341087190085056e-18
  la acabar a corrupción se ya va: 1.341087190085056e-18
  la acabar a corrupción va se ya: 1.341087190085056e-18




## El ahorcado

**Diseñe una función que sea capaz de encontrar los caracteres faltantes de una palabra. Para ello proponga una adaptación simple de la estrategia de corrección ortográfica propuesta por Norvig. La función de el ahorcado debe poder tratar con hasta 4 caracteres desconocidos en palabras de longitud arbitraria. La función debe trabajar en tiempo razonable (≈ 1 minuto en una laptop o menos). La función debe trabajar como sigue con 10 ejemplos:**

**Puede resolver este punto con una extensión muy simple de la estrategia de Norvig, o alguna forma más eficiente con distancias de edición (e.g., Levenshtein) o de subcadenas (e.g., Karp Rabin, Aho-Corasick, Tries, etc.).**

In [159]:
nltk.data.path.append('/Users/guillermo_sego/anaconda3/nltk_data/')
from nltk.corpus import words
from itertools import product
from collections import Counter

In [183]:
# Utilizamos corpus de nltk
word_list = words.words()
word_freq = Counter(word_list)

In [186]:
def hangman(secuencia, word_freq):
    if '_' not in secuencia:
        # No hay caracteres desconocidos
        return secuencia if secuencia in word_freq else None
    
    letras_alfabeto = 'abcdefghijklmnopqrstuvwxyz'
    espacios_faltantes = secuencia.count('_')
    # Generar las combinaciones
    posibles_combinaciones = product(letras_alfabeto, repeat=espacios_faltantes)
    
    # Todas las combinaciones posibles
    candidatos = []
    for combinacion in posibles_combinaciones:
        # Reemplaza '_' con letras de la combinación actual
        intento = secuencia
        for letra in combinacion:
            intento = re.sub(r'_', letra, intento, count=1)
        if intento in word_freq:
            candidatos.append(intento)

    # Ordenar candidatos por frecuencia y devolver el más frecuente
    candidatos.sort(key=lambda x: word_freq.get(x, 0), reverse=True)
    return candidatos[:3], candidatos[-3:]  # Devuelve los top 3 más y menos probables


In [189]:
# Ejemplo de uso
print(f"Para pe_p_e: ", hangman("pe_p_e", word_freq))

Para pe_p_e:  (['people'], ['people'])


**Comente brevemente como integraría un modelo de lenguaje con el modelo de Norvig para tratar de resolver errores gramaticales de más alto nivel, o errores dónde el error sea una palabra que si está en el diccionario, por ejemplo: "In the science off Maths**

Integrar un modelo de lenguaje con el modelo de corrección ortográfica de Norvig para abordar errores gramaticales de más alto nivel podría realizarse de la siguiente manera:

Primero es necesario detectar errores contextuales. Hay que utilizar el modelo de lenguaje para identificar palabras que, aunque estén escritas correctamente, son improbables en el contexto dado. Por ejemplo, en la oración "In the science off Maths", la palabra "off" es gramaticalmente incorrecta a pesar de estar bien escrita.

Luego se tienen que generar candidatos para remplazo. Para las palabras detectadas como anomalías, generar candidatos de reemplazo que tengan sentido en el contexto.

Finalmente utilizar el modelo de lenguaje para evaluar la probabilidad de la oración original y las oraciones con palabras candidatas reemplazadas, seleccionando la variante que tenga la mayor probabilidad como la corrección.

## Referencias

[1] Keselj, Vlado. "Book Review: Speech and Language Processing by Daniel Jurafsky and James H. Martin." Computational Linguistics 35.3 (2009).